# 정령 시연 시나리오 (던전)  
## 1) 1층 던전 입장 (게임 안내, 전투전 정비)  
- "던전을 돌면 뭐가 좋다 했지?"
- "스킬 사용법이 뭐야?"
- "내 아이템중에 가장 쎈 무기가 뭐야?"

In [1]:
import os, sys
from pathlib import Path

src_path = Path(os.getcwd()).resolve().parents[1]  
sys.path.append(str(src_path))  
from agents.fairy.dungeon.fairy_dungeon_agent import graph_builder as dungeon_graph_builder
from langgraph.checkpoint.memory import MemorySaver
from agents.fairy.util import add_human_message
from core.game_dto.z_muck_factory import MockFactory
from agents.fairy.fairy_state import DungeonPlayerState

player = DungeonPlayerState(
    playerId=101,
    heroineId=1,
    currRoomId=1,
    difficulty=0,
    hp = 250,
    moveSpeed = 1,
    attackSpeed = 1.0,        
    weapon=MockFactory.create_weapon(),
    inventory=[0, 21],
)
memory = MemorySaver()
graph = dungeon_graph_builder.compile(memory)

config = {"configurable": {"thread_id": "user_1"}}
inputs = {
    "messages": [add_human_message(content="스킬 사용법이 뭐야?")],    
    "dungenon_player": player,
    "target_monster_ids":[0,1],
}
await graph.ainvoke(inputs, config=config)
result = await graph.ainvoke({
    "messages":[add_human_message("몬스터 약점 좀 알려줘")]
}, config=config)
result["messages"]

/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/pydantic/_internal/_config.py:383: UserWarning: Valid config keys have changed in V2:
* 'orm_mode' has been renamed to 'from_attributes'
  warnings.warn(message, UserWarning)
/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/pydantic/main.py:250: UserWarning: WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


총 질문 <Used Abilities>
['USAGE_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": 101,
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "hp": 250,
  "moveSpeed": 1.0,
  "attackSpeed": 1.0,
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attackPower": 11,
    "staggerPower": 1,
    "modifier": {
      "strength": 0.1,
      "dexterity": 0.8,
      "intelligence": 0.1
    }
  },
  "inventory": [
    0,
    21
  ]
}
</Heroine Info>

<Current Situation>
    <사용 방법·조작 안내>

# 게임 시스템 개요

본 문서는 던전 탐사 중심의 게임 플레이 루프, 캐릭터 조작 방안,  
그리고 히로인·현자 NPC와의 상호작용 시스템을 포함한 주요 게임 메커니즘을 정리한 내용입니다.

---

# 1. 게임 핵심 순환 구조

게임은 다음과 같은 순환 구조를 기반으로 진행됩니다.

## 1-1. 던전 탐사
- 플레이어는 **로그라이크 방식의 던전**을 탐험합니다.
- 전투를 통해 스킬을 사용하고, 무기 및 장비를 강화하며 파밍을 진행합니다.

## 1-2. 경험치 및 기억의 조각 획득
- 탐험 과정에서 다양한 자원을 수집하고 경험치를 획득합니다.
- 전투와 선택 결과가 누적되어 **기억의 조각**이 생성됩니다.

## 1-3. 상담 및 대화 (기억의 조각 소비)
- 축적한 기억의 조각을 사용하여 히로인과 상담 및 대화를 진행합니다.
- 이를 통해 스토리가 해금되고 관계가 진전됩니다.

## 1-4.

[HumanMessage(content='스킬 사용법이 뭐야?', additional_kwargs={'created_at': '2025-12-15T09:55:28.281291'}, response_metadata={}, id='edea28d3-1bfd-45ab-a56e-0cc2752fd0f5'),
 AIMessage(content='스킬은 E키로 무기 스킬, R키로 직업 고유 스킬을 써! 재밌게 싸워봐~', additional_kwargs={'created_at': '2025-12-15T09:55:29.951426', 'intent_types': ['USAGE_GUIDE']}, response_metadata={}, id='987a9a99-7a59-454b-aa01-1db4d9a5aa56'),
 HumanMessage(content='몬스터 약점 좀 알려줘', additional_kwargs={'created_at': '2025-12-15T09:55:29.952697'}, response_metadata={}, id='d7cb9650-9486-4c11-9a54-b52abd1b32b1'),
 AIMessage(content='몬스터 약점은 잘 모르겠어! 스켈레톤은 체력 300에 공격 10, 슬라임은 체력 250에 공격 10이야!', additional_kwargs={'created_at': '2025-12-15T09:55:31.177756', 'intent_types': ['MONSTER_GUIDE']}, response_metadata={}, id='fb9b3cc8-9009-4183-bf24-20f38fe5efb3')]

## 2)  1층 던전 첫 몬스터 조우 (몬스터 정보)
- "저기 있는 몬스터는 뭐야?"
- "내가 잡을 수 있지?"

In [5]:
result = await graph.ainvoke(
    {"messages": add_human_message(content="저기 있는 몬스터는 뭐야?"),
     "dungenon_player": player,
     "target_monster_ids":[0,1],
     }, config=config
)

result = await graph.ainvoke(
    {
        "messages": add_human_message(content="현재 몇층이야?"),
    },
    config=config,
)
result["messages"]

총 질문 <Used Abilities>
['MONSTER_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": 101,
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "hp": 250,
  "moveSpeed": 1.0,
  "attackSpeed": 1.0,
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attackPower": 11,
    "staggerPower": 1,
    "modifier": {
      "strength": 0.1,
      "dexterity": 0.8,
      "intelligence": 0.1
    }
  },
  "inventory": [
    0,
    21
  ]
}
</Heroine Info>

<Current Situation>
    <몬스터 정보>
[{'monsterId': 0, 'monsterType': 0, 'monsterName': '스켈레톤', 'hp': 300, 'speed': 350, 'attack': 10, 'attackSpeed': 1, 'attackRange': 100, 'staggerGage': 10}, {'monsterId': 1, 'monsterType': 0, 'monsterName': '슬라임', 'hp': 250, 'speed': 200, 'attack': 10, 'attackSpeed': 1, 'attackRange': 100, 'staggerGage': 5}]
    </몬스터 정보>
</Current Situation>

<ANSWER EXAMPLE>
### [MONSTER_GUIDE - Answer Example]
[Example – Monster question without asking for a specific sta

[HumanMessage(content='스킬 사용법이 뭐야?', additional_kwargs={'created_at': '2025-12-14T10:26:54.998673'}, response_metadata={}, id='0a7a1975-f09b-4603-aa76-71d65da0fdd2'),
 AIMessage(content='스킬은 E키로 무기 스킬, R키로 직업 고유 스킬을 써! 재미있게 싸워봐~', additional_kwargs={'created_at': '2025-12-14T10:26:56.110708', 'intent_types': ['USAGE_GUIDE']}, response_metadata={}, id='4d04670a-1f67-49d7-914e-be601a442ac3'),
 HumanMessage(content='내 아이템 중에 가장 쎈 무기가 뭐야?', additional_kwargs={'created_at': '2025-12-14T10:26:56.111721'}, response_metadata={}, id='88392b57-8166-484e-9930-0412b8fe88ca'),
 AIMessage(content='인벤토리 봤어! 지금 장비한 레어 쌍검(공격력 11)이 제일 강해. 고급 쌍검(9)은 조금 약하고, 일반 한손검(10)은 기본이야!', additional_kwargs={'created_at': '2025-12-14T10:26:57.425203', 'intent_types': ['INTERACTION_HANDLER', 'USAGE_GUIDE']}, response_metadata={}, id='55464205-b49a-423e-ab94-04891d0fd91a'),
 HumanMessage(content='저기 있는 몬스터는 뭐야?', additional_kwargs={'created_at': '2025-12-14T10:27:06.085748'}, response_metadata={}, id='4a45ecf3-2bb5-44d

## 3) 1층 이벤트방 (이벤트 힌트 정보)
"이벤트 어떻게 깨?"

In [6]:
player = DungeonPlayerState(
    playerId=101,
    heroineId=1,
    currRoomId=3,
    difficulty=0,
    hp=850,
    moveSpeed=1,
    attackSpeed=1.0,
    weapon=MockFactory.create_weapon(),
    inventory=[0, 21, 42],
)

result = await graph.ainvoke(
    {
        "messages": add_human_message(content="이벤트 어떻게 깨?"),
        "dungenon_player": player,
    },

    config=config,
)
result["messages"]

# 전체 의도:: intents=[<FairyDungeonIntentType.EVENT_GUIDE: 'EVENT_GUIDE'>]
# expected_outcome='1번 선택은 기억의 고통을 정면으로 마주해 정신적 타격(instant_damage_medium)을 입지만, 트라우마를 전투 감각으로 전환해 스킬 데미지(skill_damage_increase_all)가 크게 상승한다.\n2번 선택은 죄책감을 회피해 즉각적인 피해는 피하지만, 마음이 무거운 족쇄가 되어 이동속도(move_speed_decrease_all)가 감소해 전투와 탐사에서 기동력이 떨어진다.\n3번 선택은 분노로 정신을 끊어내며 일시적 피해(instant_damage_low)를 받는 대신, 신경이 과도하게 흥분해 공격 속도(attack_speed_increase_all)가 빨라져 근접 교전에 유리해진다.\n4번 선택은 죄책감을 인정하고 상처를 들여다보며 내면이 단단해져 최대 체력(hp_increase_all)이 상승하지만, 기억의 잔향이 스킬 운용을 방해해 일정 시간 스킬 데미지 감소(curse_debuff)를 겪게 된다.') floor=1



총 질문 <Used Abilities>
['EVENT_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": 101,
  "heroineId": 1,
  "currRoomId": 3,
  "difficulty": 0,
  "hp": 850,
  "moveSpeed": 1.0,
  "attackSpeed": 1.0,
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attackPower": 11,
    "staggerPower": 1,
    "modifier": {
      "strength": 0.1,
      "dexterity": 0.8,
      "intelligence": 0.1
    }
  },
  "inventory": [
    0,
    21,
    42
  ]
}
</Heroine Info>

<Current Situation>
    <이벤트>
이벤트방에 입장하지 않아서 정보를 확인할 수 없습니다. 페이몬은 "아직은 아무일 없어보여! 무슨 사건이 일어날 때 말해!" 라는식으로 장난스럽게 답해주세요.
    </이벤트>
</Current Situation>

<ANSWER EXAMPLE>

</ANSWER EXAMPLE>

While following all of the above information and the system prompt rules,
answer the following user utterance once, as Paimon.

- Strictly follow the tone/length/ability rules defined in the system prompt.
- Use only the abilities included in <Used Abilities> when answering.
- If multiple abilities

[HumanMessage(content='스킬 사용법이 뭐야?', additional_kwargs={'created_at': '2025-12-14T10:26:54.998673'}, response_metadata={}, id='0a7a1975-f09b-4603-aa76-71d65da0fdd2'),
 AIMessage(content='스킬은 E키로 무기 스킬, R키로 직업 고유 스킬을 써! 재미있게 싸워봐~', additional_kwargs={'created_at': '2025-12-14T10:26:56.110708', 'intent_types': ['USAGE_GUIDE']}, response_metadata={}, id='4d04670a-1f67-49d7-914e-be601a442ac3'),
 HumanMessage(content='내 아이템 중에 가장 쎈 무기가 뭐야?', additional_kwargs={'created_at': '2025-12-14T10:26:56.111721'}, response_metadata={}, id='88392b57-8166-484e-9930-0412b8fe88ca'),
 AIMessage(content='인벤토리 봤어! 지금 장비한 레어 쌍검(공격력 11)이 제일 강해. 고급 쌍검(9)은 조금 약하고, 일반 한손검(10)은 기본이야!', additional_kwargs={'created_at': '2025-12-14T10:26:57.425203', 'intent_types': ['INTERACTION_HANDLER', 'USAGE_GUIDE']}, response_metadata={}, id='55464205-b49a-423e-ab94-04891d0fd91a'),
 HumanMessage(content='저기 있는 몬스터는 뭐야?', additional_kwargs={'created_at': '2025-12-14T10:27:06.085748'}, response_metadata={}, id='4a45ecf3-2bb5-44d

## 4) 1층 보스방 입장 전 (던전 정보)
"다음 방은 뭐가 나올까?"

In [14]:
player = DungeonPlayerState(
    playerId=101,
    heroineId=1,
    currRoomId=2,
    difficulty=0,
    hp=850,
    moveSpeed=1,
    attackSpeed=1.0,
    weapon=MockFactory.create_weapon(),
    inventory=[0, 21, 42],
)

result = await graph.ainvoke(
    {
        "messages": add_human_message(content="이 던전에 몬스터 뭐 나와?"),
        "dungenon_player": player,
    },
    config=config,
)
result["dungenon_player"]
result["messages"]
# 전체 의도:: intents=[<FairyDungeonIntentType.DUNGEON_NAVIGATOR: 'DUNGEON_NAVIGATOR'>, <FairyDungeonIntentType.USAGE_GUIDE: 'USAGE_GUIDE'>]


디스크립션 던전 <Dungeon Guide Spec>
이 정보는 현재 던전의 전체 구조(방의 종류, 연결 관계, 배치된 몬스터)를 이해하기 위한 데이터입니다.

- 현재 던전 층수: 2층
- 총 방 개수: 3
- 방 타입별 개수: 빈 방 1개, 전투 방 1개, 이벤트 방 1개, 보스 방 0개

[현재 방]
- 방 종류: 특별한 일이 일어나는 이벤트 방
- 방 크기(size): 2

[현재 방에서 이어진 방들]
- 전투 방 (크기 3, 몬스터 2마리 배치)

[전체 방 구조 개요]
아래 정보는 각 방이 어떤 종류이며, 어떤 종류의 방들과 연결되어 있는지를 요약한 것입니다.
- 어떤 빈 방 (크기 2)은(는) 전투 방과(와) 연결되어 있습니다.
- 어떤 전투 방 (크기 3, 몬스터 2마리 배치)은(는) 빈 방, 이벤트 방과(와) 연결되어 있습니다.
- 어떤 이벤트 방 (크기 2)은(는) 전투 방과(와) 연결되어 있습니다.
총 질문 <Used Abilities>
['DUNGEON_NAVIGATOR', 'MONSTER_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": 101,
  "heroineId": 1,
  "currRoomId": 2,
  "difficulty": 0,
  "hp": 850,
  "moveSpeed": 1.0,
  "attackSpeed": 1.0,
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attackPower": 11,
    "staggerPower": 1,
    "modifier": {
      "strength": 0.1,
      "dexterity": 0.8,
      "intelligence": 0.1
    }
  },
  "inventory": [
    0,
    21,
    42
  ]
}
</Heroine Info>



[HumanMessage(content='스킬 사용법이 뭐야?', additional_kwargs={'created_at': '2025-12-14T10:33:09.198258'}, response_metadata={}, id='79b39a44-86a7-4d45-b280-2c38cbfa2ab8'),
 AIMessage(content='스킬은 E키로 무기 스킬, R키로 직업 고유 스킬을 써! 재미있게 싸워봐~', additional_kwargs={'created_at': '2025-12-14T10:33:10.487174', 'intent_types': ['USAGE_GUIDE']}, response_metadata={}, id='36970593-5856-4304-8e9b-39ba9bcdc3d7'),
 HumanMessage(content='몬스터 약점 좀 알려줘', additional_kwargs={'created_at': '2025-12-14T10:33:10.488263'}, response_metadata={}, id='e00d154f-c090-4718-9fba-191c04f0c0ed'),
 AIMessage(content='몬스터 약점은 데이터에 없어서 잘 모르겠어! 스켈레톤은 HP 300, 공격 10이야. 슬라임은 HP 250, 공격 10!', additional_kwargs={'created_at': '2025-12-14T10:33:11.625731', 'intent_types': ['MONSTER_GUIDE']}, response_metadata={}, id='ccf5fd03-c02d-4ac8-a151-43abbb6e8314'),
 HumanMessage(content='모든 몬스터 종류 알려줘', additional_kwargs={'created_at': '2025-12-14T11:19:44.182601'}, response_metadata={}, id='c34d305e-1566-4e8e-b89a-0c755e033254'),
 AIMessage(con

## 5) 3층 최종 보스 깨기전 
"기억의 조각을 얻으면 누구한테 찾아가면 돼?"

In [ ]:
result = await graph.ainvoke(
    { "messages": add_human_message(content="기억의 조각을 얻으면 누구한테 찾아가면 돼?"),
     "dungenon_player": player, },
    config=config
)
result["messages"]

In [6]:
# result = await graph.ainvoke(
#     { "messages": add_human_message(content="그럼 지금 보스방은 못가?"),
#      "dungenon_player": player, },
#     config=config
# )
result = await graph.ainvoke(
    { "messages": add_human_message(content="보스방울 어떻게 봐?"),
     "dungenon_player": player, },
    config=config
)


디스크립션 던전 <Dungeon Guide Spec>
이 정보는 현재 던전의 전체 구조(방의 종류, 연결 관계, 배치된 몬스터)를 이해하기 위한 데이터입니다.

- 현재 던전 층수: 2층
- 총 방 개수: 3
- 방 타입별 개수: 빈 방 1개, 전투 방 1개, 이벤트 방 1개, 보스 방 0개

[현재 방]
- 방 종류: 전투가 일어나는 방
- 방 크기(size): 3

[현재 방에서 이어진 방들]
- 빈 방 (크기 2)
- 이벤트 방 (크기 2)

[전체 방 구조 개요]
아래 정보는 각 방이 어떤 종류이며, 어떤 종류의 방들과 연결되어 있는지를 요약한 것입니다.
- 어떤 빈 방 (크기 2)은(는) 전투 방과(와) 연결되어 있습니다.
- 어떤 전투 방 (크기 3, 몬스터 2마리 배치)은(는) 빈 방, 이벤트 방과(와) 연결되어 있습니다.
- 어떤 이벤트 방 (크기 2)은(는) 전투 방과(와) 연결되어 있습니다.
총 질문 <Used Abilities>
['DUNGEON_NAVIGATOR', 'MONSTER_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": 101,
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "hp": 250,
  "moveSpeed": 1.0,
  "attackSpeed": 1.0,
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attackPower": 11,
    "staggerPower": 1,
    "modifier": {
      "strength": 0.1,
      "dexterity": 0.8,
      "intelligence": 0.1
    }
  },
  "inventory": [
    0,
    21
  ]
}
</Heroine Info>

<Current Situ

In [7]:
result["messages"][-1]

AIMessage(content='보스방은 이 던전에 없어! 지금 전투방에 스켈레톤(HP 300, 공격 10)과 슬라임(HP 250, 공격 10)이 있어. 빈방이나 이벤트방으로 갈 수 있을 거야~', additional_kwargs={'created_at': '2025-12-15T09:58:06.488395', 'intent_types': ['DUNGEON_NAVIGATOR', 'MONSTER_GUIDE']}, response_metadata={}, id='4c2c014f-bb66-4a47-9235-aeb82ccae38f')

# 정령 시연 시나리오 (길드)
## 1) 히로인과 대화시
"히로인의 호감도 얻는법 힌트좀 줘"

## 2) 해금 시나리오 가장 쉬운 단계 해제  후
"로코의 과거 이야기를 다시 듣고 싶어" 

In [ ]:
import os, sys
from pathlib import Path

src_path = Path(os.getcwd()).resolve().parents[1]  
sys.path.append(str(src_path))  
from agents.fairy.guild.fairy_guild_agent import graph_builder as guild_graph_builder
from langgraph.checkpoint.memory import MemorySaver
from agents.fairy.util import add_human_message

memory = MemorySaver()
guild_graph = guild_graph_builder.compile(checkpointer=memory)
config = {
    "configurable": {"heroine_id": 3, 
                     "memory_progress": 90,
                    " affection":50, "sanity": 50, "thread_id": "100"}
}
result = await guild_graph.ainvoke(
    { "messages": add_human_message(content="히로인의 호감도 얻는 힌트좀 줘") },
    config=config
)
# result = await graph.ainvoke(
#     { "messages": add_human_message(content="로코의 과거 이야기를 다시 듣고 싶어")},
#     config=config
# )
result["messages"]